<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_whisper_medium_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kc9yy2m9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-kc9yy2m9
  Resolved https://github.com/huggingface/transformers to commit 083e36923a19650fa264c4173db2f63ab124bb27
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8075921 sha256=6e6a8b1201bad60e26dfd4e1a31508f101ed838d01288aff883b8d3a116c96e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-z9lurkd7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/

In [4]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [5]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [6]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1285
})

In [8]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=dc9e6dbdea5bf821bb6feade433bd795d509877265b54ae6052c1cb66ada6a05
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [23]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:21<00:00, 70.7MiB/s]


In [24]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [25]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [26]:
text

,0
0,takoer vano je da ta suradnja bude primjereno ...
1,dakle postoji veliki prostor za napredak europ...
2,premda je crna lista europske komisije odredil...
3,kao takav sporazum europske unije sa švicarsko...
4,kako bi se dodatno motiviralo radnike da sudje...
...,...
1280,nadam se da će program zaista polučiti pozitiv...
1281,uspjeh ipa programa nije bitan samo za države ...
1282,slobode na kojima smo izgradili europsku uniju...
1283,uvijek sam apsolutno i bezrezervno za jednakos...


In [27]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [28]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,takoer vano je da ta suradnja bude primjereno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00091552734375, 0.0013427734375, 0.00341796...",16000
1,male,dakle postoji veliki prostor za napredak europ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.000152587890625, 0.0002...",16000
2,female,premda je crna lista europske komisije odredil...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.0001220703125, -0.0008...",16000
3,male,kao takav sporazum europske unije sa švicarsko...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.00079345703125, -0.0012...",16000
4,male,kako bi se dodatno motiviralo radnike da sudje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003814697265625, 0.00299072265625, -0.0001...",16000
...,...,...,...,...,...
1280,female,nadam se da će program zaista polučiti pozitiv...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00238037109375, -0.00140380859375, 0.00238...",16000
1281,male,uspjeh ipa programa nije bitan samo za države ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0006103515625, 0.000213...",16000
1282,male,slobode na kojima smo izgradili europsku uniju...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001068115234375, -0.00189208984375, -0.000...",16000
1283,female,uvijek sam apsolutno i bezrezervno za jednakos...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.018585205078125, -0.03265380859375, -0.040...",16000


In [29]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,takoer vano je da ta suradnja bude primjereno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00091552734375, 0.0013427734375, 0.00341796...",16000
1,male,dakle postoji veliki prostor za napredak europ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.000152587890625, 0.0002...",16000
3,male,kao takav sporazum europske unije sa švicarsko...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.00079345703125, -0.0012...",16000
4,male,kako bi se dodatno motiviralo radnike da sudje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003814697265625, 0.00299072265625, -0.0001...",16000
6,male,u svojstvu izvjestiteljice u ime odbora za raz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002899169921875, 0.0015869140625, -0.001373...",16000
...,...,...,...,...,...
1277,male,sud je osnovan rezolucijom na temelju poglavlja.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000732421875, 0.001129150390625, 0.00067138...",16000
1278,male,u sutini je ovo izvjee instrument kojim se u i...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001220703125, 0.00054931640625, -0.00021362...",16000
1279,male,u takvim uvjetima nije ostalo puno prostora za...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, -0.000244140625, -9.15527...",16000
1281,male,uspjeh ipa programa nije bitan samo za države ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0006103515625, 0.000213...",16000


In [30]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
2,female,premda je crna lista europske komisije odredil...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.0001220703125, -0.0008...",16000
5,female,jedini način borbe protiv organiziranog krimin...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004852294921875, -0.0003662109375, 0.00250...",16000
9,female,s obzirom na kompleksnost i stalne inovacije u...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00042724609375, 0.000152587890625, -0.00070...",16000
20,female,potroai kau da se razlike nalaze i u automobil...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, -0.000274658203125, 0.0, ...",16000
22,female,primjerice ako dijete bude izloeno kemikaliji ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, 0.000518798828125, 0.00094...",16000
...,...,...,...,...,...
1271,female,od komisije i država članica tražimo da se ang...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, -0.000274658203125, -0.000...",16000
1275,female,mi smo najvei kontributor razvojne pomoi kako ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001220703125, 0.00067138671875, 0.00262451...",16000
1280,female,nadam se da će program zaista polučiti pozitiv...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00238037109375, -0.00140380859375, 0.00238...",16000
1283,female,uvijek sam apsolutno i bezrezervno za jednakos...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.018585205078125, -0.03265380859375, -0.040...",16000


In [31]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [32]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [35]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.38095238095238093
Processed 2 files, WER: 0.4444444444444444
Processed 3 files, WER: 0.375
Processed 4 files, WER: 0.1111111111111111
Processed 5 files, WER: 0.3125
Processed 6 files, WER: 0.4166666666666667
Processed 7 files, WER: 0.47619047619047616
Processed 8 files, WER: 0.3
Processed 9 files, WER: 0.25
Processed 10 files, WER: 0.14285714285714285
Processed 11 files, WER: 0.30434782608695654
Processed 12 files, WER: 0.32
Processed 13 files, WER: 0.43478260869565216
Processed 14 files, WER: 0.4583333333333333
Processed 15 files, WER: 0.5714285714285714
Processed 16 files, WER: 0.3488372093023256
Processed 17 files, WER: 0.24
Processed 18 files, WER: 0.3611111111111111
Processed 19 files, WER: 0.2
Processed 20 files, WER: 0.3333333333333333
Processed 21 files, WER: 0.5405405405405406
Processed 22 files, WER: 0.6363636363636364
Processed 23 files, WER: 0.14285714285714285
Processed 24 files, WER: 0.14814814814814814
Processed 25 files, WER: 0.46875
Processed 

In [36]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.35294117647058826
Processed 2 files, WER: 0.2
Processed 3 files, WER: 0.2777777777777778
Processed 4 files, WER: 0.2777777777777778
Processed 5 files, WER: 0.65
Processed 6 files, WER: 0.17391304347826086
Processed 7 files, WER: 0.46153846153846156
Processed 8 files, WER: 0.48936170212765956
Processed 9 files, WER: 0.47368421052631576
Processed 10 files, WER: 0.1875
Processed 11 files, WER: 0.10526315789473684
Processed 12 files, WER: 0.4
Processed 13 files, WER: 0.23076923076923078
Processed 14 files, WER: 0.41935483870967744
Processed 15 files, WER: 0.2
Processed 16 files, WER: 0.07142857142857142
Processed 17 files, WER: 0.3333333333333333
Processed 18 files, WER: 0.3333333333333333
Processed 19 files, WER: 0.36
Processed 20 files, WER: 0.21052631578947367
Processed 21 files, WER: 0.45454545454545453
Processed 22 files, WER: 0.37142857142857144
Processed 23 files, WER: 0.7142857142857143
Processed 24 files, WER: 0.2857142857142857
Processed 25 files, WER: 0